## benchmark (Polyglot)

In [ ]:
// // test

open testing

()



In [ ]:
inl (/@) x = listm'.(/@) x

()



In [ ]:
inl am'_average forall el {number}. (array : a _ el) : el =
    $"!array |> Array.average"

inl am'_parallel_map forall dim el el'. (fn : el -> el') (array : a dim el) : a dim el' =
    $"!array |> Array.Parallel.map !fn"

inl am'_sort_by forall dim el. (fn : el -> _) (array : a dim el) : a dim el =
    $"!array |> Array.sortBy !fn"

inl am'_sort_descending forall dim el. (array : a dim el) : a dim el =
    $"!array |> Array.sortDescending"

inl am'_transpose forall dim el. (array : a dim (a dim el)) : a dim (a dim el) =
    $"!array |> Array.transpose"

inl am'_try_item forall dim el. (i : i32) (array : a dim el) : option el =
    $"!array |> Array.tryItem !i" |> optionm'.from_fsharp

nominal console_color = $"System.ConsoleColor"

inl console_reset_color () : () =
    run_target function
        | Fsharp => fun () => $"System.Console.ResetColor ()"
        | _ => fun () => ()

inl console_set_foreground_color (color : console_color) : () =
    run_target function
        | Fsharp => fun () => $"System.Console.ForegroundColor <- !color"
        | _ => fun () => ()

inl gc_collect () =
    run_target function
        | Fsharp => fun () => $"System.GC.Collect ()"
        | _ => fun () => ()

nominal map k v = $"Map<`k, `v>"

inl map_item forall k v. (k : k) (map : map k v) : v =
    $"!map.[!k]"

inl map_of_array forall k v. (array : a _ (k * v)) : map k v =
    $"!array |> Array.map (fun (struct (a, b)) -> a, b) |> Map.ofArray"

inl printfn x =
    console.write_line x

nominal stopwatch = $"System.Diagnostics.Stopwatch"

inl stopwatch () : stopwatch =
    $"`stopwatch" ()

inl stopwatch_elapsed_milliseconds (stopwatch : stopwatch) : i64 =
    $"!stopwatch.ElapsedMilliseconds"

inl stopwatch_start (stopwatch : stopwatch) : () =
    $"!stopwatch.Start ()"

inl string_concat (a : string) (b : a _ _) : string =
    $"!b |> String.concat !a"

inl string_join (concat : string) (s : a _ string) : string =
    $"System.String.Join (!concat, !s)"


inl string_pad_right (pad : i32) (s : string) : string =
    $"!s.PadRight !pad"

()



## test_case_result

In [ ]:
type test_case_result =
    {
        Input : string
        Expected : string
        Result : string
        TimeList : a u64 i64
    }

()



## run

In [ ]:
inl run forall input expected.
    count
    (solutions : list (string * (input -> expected)))
    ((input, expected) : (input * expected))
    : test_case_result
    =
    inl input_str = input |> format_debug

    printfn ""
    printfn ($"$\"Solution: {!input_str}  \"" : string)

    inl performance_invoke (fn : () -> expected) =
        gc_collect ()
        inl stopwatch = stopwatch ()
        stopwatch |> stopwatch_start
        inl time1 = stopwatch |> stopwatch_elapsed_milliseconds
        inl result : expected =
            am'.init_series 0 count 1i32
            |> am'_parallel_map fun _n => fn ()
            |> am'.last
        inl time2 = (stopwatch |> stopwatch_elapsed_milliseconds) - time1
        result, time2

    inl results_with_time : a u64 _ =
        solutions
        |> listm'.indexed
        |> listm.toArray
        |> am.map fun ((i : i32), (test_name, solution)) =>
            inl result, time = performance_invoke fun () => solution input
            printfn ($"$\"Test case {!i + 1}. {!test_name}. Time: {!time}  \"" : string)
            result, time

    match results_with_time |> am.map fst with
    | array when length array <= 1 => ()
    | array when array |> am.forall' ((=) (index array 0)) => ()
    | results => failwith ($"$\"Challenge error: {!results}\"" : string)

    {
        Input = input_str
        Expected = expected |> format_debug
        Result = results_with_time |> am.map fst |> fun array => index array 0 |> format_debug
        TimeList = results_with_time |> am.map snd
    }

()



## run_all

In [ ]:
inl run_all forall input expected.
    test_name
    count
    (solutions : list (string * (input -> expected)))
    test_cases
    =
    printfn ""
    printfn ""
    printfn ($"$\"Test: {!test_name}\"" : string)
    test_cases
    |> listm.toArray
    |> am.map (run count solutions)

()



## sort_result_list

In [ ]:
inl sort_result_list results =
    inl table =
        inl rows =
            results
            |> am.map fun (result : test_case_result) =>
                inl best =
                    result.TimeList
                    |> am'.indexed
                    |> am.map fun (i, time) =>
                        i + 1i64, time
                    |> am'_sort_by snd
                    |> fun array => index array 0i32
                    |> format_debug
                inl row =
                    [
                        result.Input
                        result.Expected
                        result.Result
                        best
                    ]
                inl color : option console_color =
                    match result.Expected = result.Result with
                    | true => Some $"`console_color.DarkGreen"
                    | false => Some $"`console_color.DarkRed"
                row, color

        inl header =
            [
                [
                    "Input"
                    "Expected"
                    "Result"
                    "Best"
                ]
                [
                    "---"
                    "---"
                    "---"
                    "---"
                ]
            ]
            |> listm.map fun row => row, None
            |> listm.toArray
        rows |> am.append header

    inl formattedTable =
        inl lengthMap : map i32 i64 =
            table
            |> am.map (fst >> listm.toArray)
            |> am'_transpose
            |> am.map fun column =>
                column
                |> am.map sm.length
                |> am'_sort_descending
                |> am'_try_item 0i32
                |> optionm'.default_value 0i64
            |> am'.indexed
            |> fun (x : a i32 _) => x
            |> map_of_array
        table
        |> am.map fun (row, color) =>
            inl newRow =
                row
                |> listm'.indexed
                |> listm.map fun (i, cell) =>
                    cell |> string_pad_right (lengthMap |> map_item i |> conv)
                |> listm.toArray
            newRow, color

    printfn ""
    formattedTable
    |> am.iter fun ((row : a i32 string), color) =>
        match color with
        | Some color => color |> console_set_foreground_color
        | None => console_reset_color ()

        printfn (row |> string_join "\t| ")

        console_reset_color ()

    inl averages : a u64 _ =
        results
        |> am.map fun result =>
            result.TimeList
            |> am.map ($"float" : i64 -> f64)
        |> am'_transpose
        |> am.map am'_average
        |> am.map ($"int64" : f64 -> i64)
        |> am'.indexed

    printfn ""
    printfn "Average Ranking  "
    averages
    |> am'_sort_by snd
    |> am.iter fun ((i : i32), avg) =>
        printfn ($"$\"Test case %d{!i + 1}. Average Time: %A{!avg}  \"" : string)

()



In [ ]:
// // test

inl is_fast () =
    false

()



## empty2Tests

Test: Empty2

Solution: (a, a)  
Test case 1. A. Time: 59L

Solution: (a, a)  
Test case 1. A. Time: 53L

Input   | Expected        | Result  | Best
---     | ---             | ---     | ---
(a, a)  | a               | a       | (1, 59)
(a, a)  | a               | a       | (1, 53)

Averages  
Test case 1. Average Time: 56L

Ranking  
Test case 1. Average Time: 56L

In [ ]:
// // test

inl get_solutions () =
    [
        "A",
        fun (a, _b) =>
            a

        "B",
        fun (_a, b) =>
            b
    ]

inl rec empty_2_tests () =
    inl test_cases = [
        ("a", "a"), "a"
        ("b", "b"), "b"
    ]

    inl solutions = get_solutions ()

    // inl is_fast () = true

    inl count =
        if is_fast ()
        then 1000i32
        else 2000000i32

    run_all (nameof empty_2_tests) count solutions test_cases
    |> sort_result_list

empty_2_tests ()

type UH0 =
    | UH0_0 of string * string * string * UH0
    | UH0_1
and Mut0 = {mutable l0 : uint64}
and UH1 =
    | UH1_0 of int32 * string * (struct (string * string) -> string) * UH1
    | UH1_1
and Mut1 = {mutable l0 : int32}
and UH2 =
    | UH2_0 of string * UH2
    | UH2_1
and [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : System.ConsoleColor
and UH3 =
    | UH3_0 of int64 * int64 * UH3
    | UH3_1
and Mut2 = {mutable l0 : uint64; mutable l1 : UH3; mutable l2 : int64}
and UH4 =
    | UH4_0 of UH2 * US0 * UH4
    | UH4_1
and [<Struct>] US1 =
    | US1_0
    | US1_1 of f1_0 : int64
and UH5 =
    | UH5_0 of int32 * int64 * UH5
    | UH5_1
and Mut3 = {mutable l0 : uint64; mutable l1 : UH5; mutable l2 : int32}
and UH6 =
    | UH6_0 of int32 * string * UH6
    | UH6_1
let rec method2 (v0 : UH0, v1 : uint64) : uint64 =
    match v0 with
    | UH0_0(v2, v3, v4, v5) -> (* Cons *)
        let v6 : uint64 = v1 + 1UL
        method2(v5, v6)
    | UH0_1 -> (* Nil *)
        v1
and method

## emptyTests

Test: Empty

Solution: 0  
Test case 1. A. Time: 61L

Solution: 2  
Test case 1. A. Time: 62L

Solution: 5  
Test case 1. A. Time: 70L

Input   | Expected        | Result  | Best
---     | ---             | ---     | ---
0       | 0               | 0       | (1, 61)
2       | 2               | 2       | (1, 62)
5       | 5               | 5       | (1, 70)

Averages  
Test case 1. Average Time: 64L

Ranking  
Test case 1. Average Time: 64L

In [ ]:
// // test

inl get_solutions () =
    [
        "A",
        fun n =>
            n + 1f64
    ]

inl rec empty_1_tests () =
    inl test_cases = [
        0, 1
        2, 3
        5, 6
    ]

    inl solutions = get_solutions ()

    // inl is_fast () = true

    inl count =
        if is_fast ()
        then 1000i32
        else 2000000i32

    run_all (nameof empty_1_tests) count solutions test_cases
    |> sort_result_list

empty_1_tests ()

type UH0 =
    | UH0_0 of float * float * UH0
    | UH0_1
and Mut0 = {mutable l0 : uint64}
and UH1 =
    | UH1_0 of int32 * string * (float -> float) * UH1
    | UH1_1
and Mut1 = {mutable l0 : int32}
and UH2 =
    | UH2_0 of string * UH2
    | UH2_1
and [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : System.ConsoleColor
and UH3 =
    | UH3_0 of int64 * int64 * UH3
    | UH3_1
and Mut2 = {mutable l0 : uint64; mutable l1 : UH3; mutable l2 : int64}
and UH4 =
    | UH4_0 of UH2 * US0 * UH4
    | UH4_1
and [<Struct>] US1 =
    | US1_0
    | US1_1 of f1_0 : int64
and UH5 =
    | UH5_0 of int32 * int64 * UH5
    | UH5_1
and Mut3 = {mutable l0 : uint64; mutable l1 : UH5; mutable l2 : int32}
and UH6 =
    | UH6_0 of int32 * string * UH6
    | UH6_1
let rec method2 (v0 : UH0, v1 : uint64) : uint64 =
    match v0 with
    | UH0_0(v2, v3, v4) -> (* Cons *)
        let v5 : uint64 = v1 + 1UL
        method2(v4, v5)
    | UH0_1 -> (* Nil *)
        v1
and method3 (v0 : (struct (float * float) [])